In [ ]:
"""
DEEP NEURAL NETWORKS - ASSIGNMENT 2: CNN FOR IMAGE CLASSIFICATION
Convolutional Neural Networks: Custom Implementation vs Transfer Learning
"""

In [ ]:
def check_submission_readiness():
    """
    Check if the notebook is ready for submission
    Verifies all requirements are met
    """
    print("=" * 70)
    print("🔍 SUBMISSION READINESS CHECK")
    print("=" * 70)
    
    issues = []
    warnings = []
    
    # Check student info
    try:
        if "TODO" in dataset_name or dataset_name == "":
            issues.append("❌ Dataset name not filled")
        else:
            print("✅ Dataset name filled")
    except:
        issues.append("❌ Dataset name variable not defined")
    
    # Check if models are trained
    try:
        if custom_cnn_accuracy == 0.0:
            issues.append("❌ Custom CNN not trained (accuracy is 0.0)")
        else:
            print(f"✅ Custom CNN trained (accuracy: {custom_cnn_accuracy:.4f})")
    except:
        issues.append("❌ Custom CNN metrics not calculated")
    
    try:
        if tl_accuracy == 0.0:
            issues.append("❌ Transfer Learning not trained (accuracy is 0.0)")
        else:
            print(f"✅ Transfer Learning trained (accuracy: {tl_accuracy:.4f})")
    except:
        issues.append("❌ Transfer Learning metrics not calculated")
    
    # Check GAP usage
    try:
        custom_summary = [layer.__class__.__name__ for layer in custom_cnn.layers]
        if 'GlobalAveragePooling2D' in custom_summary:
            print("✅ Custom CNN uses Global Average Pooling")
        else:
            issues.append("❌ Custom CNN missing Global Average Pooling")
    except:
        issues.append("❌ Cannot verify Custom CNN architecture")
    
    try:
        tl_summary = [layer.__class__.__name__ for layer in transfer_model.layers]
        if 'GlobalAveragePooling2D' in str(tl_summary):
            print("✅ Transfer Learning uses Global Average Pooling")
        else:
            issues.append("❌ Transfer Learning missing Global Average Pooling")
    except:
        issues.append("❌ Cannot verify Transfer Learning architecture")
    
    # Check data split
    if train_test_ratio == "TODO: 90/10 OR 85/15":
        issues.append("❌ Train/test ratio not documented")
    else:
        print(f"✅ Train/test split documented: {train_test_ratio}")
    
    # Final verdict
    print("\n" + "=" * 70)
    if issues:
        print("⚠️ ISSUES FOUND - FIX BEFORE SUBMISSION:")
        for issue in issues:
            print(f"  {issue}")
    else:
        print("🎉 ALL CHECKS PASSED!")
        print("✅ Notebook appears ready for submission")
    
    if warnings:
        print("\n⚡ WARNINGS:")
        for warning in warnings:
            print(f"  {warning}")
    
    print("\n📝 Final Steps:")
    print("  1. Run 'Restart & Run All' from Kernel menu")
    print("  2. Verify all outputs are visible")
    print("  3. Save notebook")
    print("  4. Rename file to: <BITS_ID>_cnn_assignment.ipynb")
    print("  5. Submit ONLY the .ipynb file")
    print("=" * 70)

# This will be called at the end of the notebook
# Uncomment before submission to run the check

## 📋 PRE-SUBMISSION CHECKLIST

Run this cell before submission to ensure everything is ready:

```python
# Uncomment and run before submission
# check_submission_readiness()
```

This will verify:
- ✅ All required metadata is filled
- ✅ Both models have been trained
- ✅ All metrics are calculated
- ✅ All outputs are visible
- ✅ Global Average Pooling is used
- ✅ File naming is correct

In [ ]:
"""
STUDENT INFORMATION (REQUIRED - DO NOT DELETE)

BITS ID: [Enter your BITS ID here - e.g., 2025AA05036]
Name: [Enter your full name here - e.g., JOHN DOE]
Email: [Enter your email]
Date: [Submission date]

"""

In [ ]:
"""
ASSIGNMENT OVERVIEW

This assignment requires you to implement and compare two CNN approaches for 
image classification:
1. Custom CNN architecture using Keras/PyTorch
2. Transfer Learning using pre-trained models (ResNet/VGG)

Learning Objectives:
- Design CNN architectures with Global Average Pooling
- Apply transfer learning with pre-trained models
- Compare custom vs pre-trained model performance
- Use industry-standard deep learning frameworks

IMPORTANT: Global Average Pooling (GAP) is MANDATORY for both models.
DO NOT use Flatten + Dense layers in the final architecture.

"""

In [ ]:
"""
 IMPORTANT SUBMISSION REQUIREMENTS - STRICTLY ENFORCED 

1. FILENAME FORMAT: <BITS_ID>_cnn_assignment.ipynb
   Example: 2025AA05036_cnn_assignment.ipynb
    Wrong filename = Automatic 0 marks

2. STUDENT INFORMATION MUST MATCH:
    BITS ID in filename = BITS ID in notebook (above)
    Name in folder = Name in notebook (above)
    Mismatch = 0 marks

3. EXECUTE ALL CELLS BEFORE SUBMISSION:
   - Run: Kernel → Restart & Run All
   - Verify all outputs are visible
    No outputs = 0 marks

4. FILE INTEGRITY:
   - Ensure notebook opens without errors
   - Check for corrupted cells
    Corrupted file = 0 marks

5. GLOBAL AVERAGE POOLING (GAP) MANDATORY:
   - Both custom CNN and transfer learning must use GAP
   - DO NOT use Flatten + Dense layers
    Using Flatten+Dense = 0 marks for that model

6. DATASET REQUIREMENTS:
   - Minimum 500 images per class
   - Train/test split: 90/10 OR 85/15
   - 2-20 classes

7. USE KERAS OR PYTORCH:
   - Use standard model.fit() or training loops
   - Do NOT implement convolution from scratch

8. FILE SUBMISSION:
   - Submit ONLY the .ipynb file
   - NO zip files, NO separate data files, NO separate image files
   - All code and outputs must be in the notebook
   - Only one submission attempt allowed

"""

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from collections import Counter
import time
import json
import os

In [ ]:
# Deep learning frameworks
import tensorflow as tf
from tensorflow import keras


# For image processingprint(f"TensorFlow version: {tf.__version__}")

from PIL import Image
import cv2

In [ ]:
# Additional libraries for Kaggle dataset loading
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
import zipfile
import shutil
from pathlib import Path

In [ ]:
# Progress tracking utilities
from IPython.display import clear_output, display, HTML

class ProgressTracker:
    """Track and display training progress"""
    
    def __init__(self, total_models=2):
        self.total_models = total_models
        self.completed_models = 0
        self.current_model = None
        self.start_time = time.time()
    
    def start_model(self, model_name):
        """Start tracking a new model"""
        self.current_model = model_name
        print(f"\n{'='*70}")
        print(f"🚀 Starting: {model_name}")
        print(f"{'='*70}")
    
    def complete_model(self, model_name, training_time):
        """Mark a model as complete"""
        self.completed_models += 1
        print(f"\n✅ Completed: {model_name} in {training_time:.2f}s")
        print(f"📊 Progress: {self.completed_models}/{self.total_models} models completed")
    
    def show_overall_progress(self):
        """Show overall progress"""
        elapsed = time.time() - self.start_time
        print(f"\n⏱️ Total elapsed time: {elapsed:.2f}s")
        print(f"📈 Overall progress: {(self.completed_models/self.total_models)*100:.1f}%")

progress_tracker = ProgressTracker()
print("✅ Progress tracker initialized")

In [ ]:
# Set random seeds for reproducibility
def set_seeds(seed=42):
    """Set all random seeds for reproducibility"""
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    print(f"✅ Random seeds set to {seed}")

set_seeds(config.random_seed)

# Setup GPU if available
def setup_gpu():
    """Configure GPU settings for optimal performance"""
    gpus = tf.config.list_physical_devices('GPU')
    
    if gpus:
        try:
            # Enable memory growth to prevent TF from allocating all GPU memory
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            
            # Enable mixed precision for faster training
            if config.use_mixed_precision:
                from tensorflow.keras import mixed_precision
                policy = mixed_precision.Policy('mixed_float16')
                mixed_precision.set_global_policy(policy)
                print(f"✅ Mixed precision enabled (GPU detected)")
            
            print(f"✅ GPU configured: {len(gpus)} GPU(s) available")
            for i, gpu in enumerate(gpus):
                print(f"   GPU {i}: {gpu.name}")
        except RuntimeError as e:
            print(f"⚠️ GPU configuration error: {e}")
    else:
        print("ℹ️ No GPU detected - using CPU")
        print("   ⚡ Tip: Training will be slower. Consider using Google Colab for GPU access.")

setup_gpu()

In [ ]:
from datetime import datetime
import pickle
from tqdm.auto import tqdm

class TrainingManager:
    """Manages checkpointing, logging, and resumable training"""
    
    def __init__(self, model_name, config):
        self.model_name = model_name
        self.config = config
        self.checkpoint_path = Path(config.checkpoint_dir) / model_name
        self.logs_path = Path(config.logs_dir) / model_name
        
        # Create directories
        self.checkpoint_path.mkdir(parents=True, exist_ok=True)
        self.logs_path.mkdir(parents=True, exist_ok=True)
        
        # Training state
        self.training_state = {
            'epoch': 0,
            'history': {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []},
            'best_val_loss': float('inf'),
            'training_time': 0.0,
            'completed': False
        }
        
        # Load existing checkpoint if available
        self.load_checkpoint()
    
    def load_checkpoint(self):
        """Load the latest checkpoint if it exists"""
        checkpoint_file = self.checkpoint_path / 'training_state.pkl'
        
        if checkpoint_file.exists():
            try:
                with open(checkpoint_file, 'rb') as f:
                    self.training_state = pickle.load(f)
                print(f"✅ Loaded checkpoint for {self.model_name}")
                print(f"   Last completed epoch: {self.training_state['epoch']}")
                print(f"   Best val_loss: {self.training_state['best_val_loss']:.4f}")
                return True
            except Exception as e:
                print(f"⚠️ Could not load checkpoint: {e}")
                return False
        else:
            print(f"ℹ️ No checkpoint found for {self.model_name} - starting fresh")
            return False
    
    def save_checkpoint(self, model, epoch, history, metrics):
        """Save model checkpoint and training state"""
        if not self.config.save_checkpoints:
            return
        
        # Update training state
        self.training_state['epoch'] = epoch
        self.training_state['history'] = history
        
        # Save training state
        with open(self.checkpoint_path / 'training_state.pkl', 'wb') as f:
            pickle.dump(self.training_state, f)
        
        # Save model weights
        model_file = self.checkpoint_path / f'{self.model_name}_epoch_{epoch}.h5'
        model.save_weights(str(model_file))
        
        # Save best model
        val_loss = metrics.get('val_loss', float('inf'))
        if val_loss < self.training_state['best_val_loss']:
            self.training_state['best_val_loss'] = val_loss
            best_model_file = self.checkpoint_path / f'{self.model_name}_best.h5'
            model.save_weights(str(best_model_file))
            print(f"💾 Saved best model (val_loss: {val_loss:.4f})")
    
    def should_resume(self):
        """Check if training should resume from checkpoint"""
        return (self.training_state['epoch'] > 0 and 
                not self.training_state['completed'])
    
    def get_initial_epoch(self):
        """Get the epoch number to resume from"""
        return self.training_state['epoch']
    
    def mark_completed(self):
        """Mark training as completed"""
        self.training_state['completed'] = True
        with open(self.checkpoint_path / 'training_state.pkl', 'wb') as f:
            pickle.dump(self.training_state, f)
    
    def log_metrics(self, epoch, metrics, mode='train'):
        """Log metrics to file"""
        if not self.config.log_metrics:
            return
        
        log_file = self.logs_path / f'{mode}_metrics.csv'
        
        # Create header if file doesn't exist
        if not log_file.exists():
            with open(log_file, 'w') as f:
                f.write('timestamp,epoch,' + ','.join(metrics.keys()) + '\n')
        
        # Append metrics
        with open(log_file, 'a') as f:
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            values = ','.join([str(v) for v in metrics.values()])
            f.write(f'{timestamp},{epoch},{values}\n')

# Test the training manager
print("\n📦 Training Manager initialized")
print(f"   Checkpoint directory: {config.checkpoint_dir}")
print(f"   Logs directory: {config.logs_dir}")

## 🔧 UTILITY FUNCTIONS
Helper functions for checkpointing, logging, and progress tracking

In [ ]:
from dataclasses import dataclass
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

@dataclass
class Config:
    """Centralized configuration for the entire project"""
    
    # ============ PATHS ============
    data_dir: str = './data'
    checkpoint_dir: str = './checkpoints'
    logs_dir: str = './logs'
    results_dir: str = './results'
    
    # ============ DATASET ============
    dataset_slug: str = 'maedemaftouni/large-covid19-ct-slice-dataset'
    img_size: Tuple[int, int] = (224, 224)
    train_test_split: float = 0.10  # 90/10 split
    
    # ============ TRAINING - CUSTOM CNN ============
    custom_batch_size: int = 32
    custom_epochs: int = 20
    custom_learning_rate: float = 0.001
    custom_optimizer: str = 'Adam'
    
    # ============ TRAINING - TRANSFER LEARNING ============
    tl_batch_size: int = 32
    tl_epochs: int = 15
    tl_learning_rate: float = 0.0001
    tl_optimizer: str = 'Adam'
    pretrained_model: str = 'ResNet50'  # Options: 'ResNet50', 'VGG16', 'MobileNetV2'
    
    # ============ OPTIMIZATION ============
    use_mixed_precision: bool = True  # For faster training on compatible GPUs
    cache_dataset: bool = True  # Cache preprocessed data
    prefetch_buffer: int = tf.data.AUTOTUNE
    
    # ============ CHECKPOINTING ============
    save_checkpoints: bool = True
    checkpoint_frequency: int = 5  # Save every N epochs
    save_best_only: bool = True
    
    # ============ LOGGING ============
    verbose_training: int = 1  # 0=silent, 1=progress bar, 2=one line per epoch
    log_metrics: bool = True
    
    # ============ REPRODUCIBILITY ============
    random_seed: int = 42
    
    # ============ DEBUG MODE ============
    debug_mode: bool = False  # Set True for faster debugging
    debug_samples: int = 500  # Use limited samples in debug mode
    debug_epochs: int = 2
    
    def __post_init__(self):
        """Adjust settings based on debug mode"""
        if self.debug_mode:
            print("⚠️ DEBUG MODE ENABLED - Using reduced dataset and epochs")
            self.custom_epochs = self.debug_epochs
            self.tl_epochs = self.debug_epochs
            self.custom_batch_size = min(16, self.custom_batch_size)
            self.tl_batch_size = min(16, self.tl_batch_size)

# Initialize configuration
config = Config()

print("✅ Configuration initialized")
print(f"   Debug Mode: {config.debug_mode}")
print(f"   Image Size: {config.img_size}")
print(f"   Custom CNN - Batch: {config.custom_batch_size}, Epochs: {config.custom_epochs}")
print(f"   Transfer Learning - Batch: {config.tl_batch_size}, Epochs: {config.tl_epochs}")

## 📋 CENTRALIZED CONFIGURATION
All hyperparameters, paths, and settings are defined here for easy modification

## 🚀 QUICK START GUIDE

### First Time Setup:
1. **Set Kaggle Credentials**: Place `kaggle.json` in `~/.kaggle/` directory
2. **Adjust Configuration**: Modify the `Config` class if needed (debug mode, epochs, batch size, etc.)
3. **Run All Cells**: Execute from top to bottom

### Resuming After Interruption:
1. **Don't Panic!** Your progress is saved
2. **Just Run All**: The training will automatically resume from the last checkpoint
3. **Check Logs**: Look for "🔄 Resuming training from epoch X" message

### Debug Mode (Fast Iteration):
```python
config.debug_mode = True  # Uses fewer samples and epochs
```

### Production Mode:
```python
config.debug_mode = False  # Full dataset and training
```

### Changing Models:
```python
config.pretrained_model = 'ResNet50'  # or 'VGG16', 'MobileNetV2'
```

### Key Directories Created:
- `./data/` - Downloaded dataset
- `./checkpoints/` - Model weights and training state
- `./logs/` - Training metrics and TensorBoard logs
- `./results/` - Final outputs

### ⚠️ Important Notes:
- **Do NOT** delete checkpoint files during training
- **Training can be interrupted** safely with Ctrl+C
- **Kernel can be restarted** - training will resume
- **All outputs are preserved** for submission


================================================================================
PART 1: DATASET LOADING AND EXPLORATION
================================================================================

Instructions:
1. Choose ONE dataset from the allowed list
2. Load and explore the data
3. Fill in ALL required metadata fields below
4. Provide justification for your primary metric choice


## 1.1 Dataset Metadata

In [ ]:
# REQUIRED: Fill in these metadata fields
dataset_name = "Large COVID-19 CT Slice Dataset"
dataset_source = "Kaggle - maedemaftouni/large-covid19-ct-slice-dataset"
n_samples = 0  # Will be updated after loading
n_classes = 2  # COVID and Non-COVID
samples_per_class = "Will be calculated after loading"
image_shape = [224, 224, 3]  # [height, width, channels]
problem_type = "classification"

In [ ]:
# Primary metric selection
primary_metric = "recall"
metric_justification = """
Recall is chosen as the primary metric for COVID-19 detection because minimizing false negatives 
is critical - we want to ensure that COVID-positive cases are not missed, even at the cost of 
some false positives which can be verified with additional tests.
"""

In [ ]:
print("\n" + "="*70)
print("DATASET INFORMATION")
print("="*70)
print(f"Dataset: {dataset_name}")
print(f"Source: {dataset_source}")
print(f"Total Samples: {n_samples}")
print(f"Number of Classes: {n_classes}")
print(f"Samples per Class: {samples_per_class}")
print(f"Image Shape: {image_shape}")
print(f"Primary Metric: {primary_metric}")
print(f"Metric Justification: {metric_justification}")
print("="*70)

## 1.2 Data Exploration

### 1.2.1 Setup Kaggle API
First, ensure you have your Kaggle API credentials set up. You need to:
1. Go to Kaggle → Account → API → Create New API Token
2. This downloads kaggle.json
3. Place it in: `~/.kaggle/kaggle.json` (Linux/Mac) or `C:\Users\<YourUsername>\.kaggle\kaggle.json` (Windows)

In [ ]:
# Configure Kaggle API
api = KaggleApi()
api.authenticate()

# Set paths using config
data_dir = Path(config.data_dir)
data_dir.mkdir(exist_ok=True)

print("✅ Kaggle API authenticated successfully!")
print(f"   Data directory: {data_dir}")

### 1.2.2 Download Dataset from Kaggle

In [ ]:
# Download the dataset (only if not already downloaded)
dataset_slug = config.dataset_slug
download_path = str(data_dir)

# Check if dataset already exists
dataset_exists = any(data_dir.iterdir()) if data_dir.exists() else False

if not dataset_exists:
    print(f"📥 Downloading dataset: {dataset_slug}")
    print(f"   Destination: {download_path}")
    
    # Download dataset
    api.dataset_download_files(dataset_slug, path=download_path, unzip=True)
    print("✅ Dataset downloaded successfully!")
else:
    print(f"ℹ️ Dataset already exists at {download_path}")

print(f"\n📁 Files in {download_path}:")
for item in os.listdir(download_path):
    print(f"   - {item}")

### 1.2.3 Explore Dataset Structure

In [ ]:
# Explore dataset structure
def explore_dataset_structure(base_path):
    """Explore and print the dataset directory structure"""
    base_path = Path(base_path)
    
    print("Dataset Structure:")
    print("=" * 70)
    
    # Find all subdirectories
    subdirs = [d for d in base_path.iterdir() if d.is_dir()]
    
    if not subdirs:
        print(f"No subdirectories found in {base_path}")
        print("\nListing all items:")
        for item in base_path.iterdir():
            print(f"  {item.name} - {'Dir' if item.is_dir() else 'File'}")
    else:
        for subdir in subdirs:
            # Count images in each subdirectory
            image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
            image_files = [f for f in subdir.iterdir() 
                          if f.is_file() and f.suffix.lower() in image_extensions]
            
            print(f"\n{subdir.name}/")
            print(f"  Number of images: {len(image_files)}")
            
            if image_files:
                # Show first few filenames
                print(f"  Sample files:")
                for img in image_files[:3]:
                    print(f"    - {img.name}")
    
    return subdirs

subdirs = explore_dataset_structure(data_dir)
print("\n" + "=" * 70)

### 1.2.4 Load Image Data

In [ ]:
# Load image paths and labels
def load_image_paths_and_labels(base_path):
    """
    Load all image paths and their corresponding labels
    Returns: image_paths (list), labels (list), class_names (list)
    """
    base_path = Path(base_path)
    image_paths = []
    labels = []
    class_names = []
    
    # Image extensions to look for
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']
    
    # Get all subdirectories (each represents a class)
    class_dirs = sorted([d for d in base_path.iterdir() if d.is_dir()])
    
    if not class_dirs:
        print("Warning: No class directories found!")
        # Try to find images directly in the base path
        all_images = [f for f in base_path.iterdir() 
                     if f.is_file() and f.suffix.lower() in image_extensions]
        if all_images:
            print(f"Found {len(all_images)} images in base directory")
            # Try to infer classes from filenames
            return all_images, [0] * len(all_images), ['unknown']
    
    class_names = [d.name for d in class_dirs]
    print(f"Found {len(class_names)} classes: {class_names}")
    
    # Load images from each class directory
    for class_idx, class_dir in enumerate(class_dirs):
        class_images = [f for f in class_dir.iterdir() 
                       if f.is_file() and f.suffix.lower() in image_extensions]
        
        print(f"  {class_names[class_idx]}: {len(class_images)} images")
        
        for img_path in class_images:
            image_paths.append(str(img_path))
            labels.append(class_idx)
    
    print(f"\nTotal images loaded: {len(image_paths)}")
    
    return image_paths, labels, class_names

# Load the data
image_paths, labels, class_names = load_image_paths_and_labels(data_dir)

# Update metadata
n_samples = len(image_paths)
n_classes = len(class_names)

# Calculate samples per class
from collections import Counter
label_counts = Counter(labels)
samples_per_class = f"min: {min(label_counts.values())}, max: {max(label_counts.values())}, avg: {n_samples // n_classes}"

print(f"\nClass distribution:")
for idx, class_name in enumerate(class_names):
    count = label_counts[idx]
    percentage = (count / n_samples) * 100
    print(f"  {class_name}: {count} images ({percentage:.2f}%)")

### 1.2.5 Visualize Sample Images

In [ ]:
# Visualize sample images from each class
def visualize_samples(image_paths, labels, class_names, samples_per_class=5):
    """Display sample images from each class"""
    n_classes = len(class_names)
    
    fig, axes = plt.subplots(n_classes, samples_per_class, 
                            figsize=(samples_per_class * 3, n_classes * 3))
    
    if n_classes == 1:
        axes = axes.reshape(1, -1)
    
    for class_idx in range(n_classes):
        # Get images for this class
        class_image_indices = [i for i, label in enumerate(labels) if label == class_idx]
        
        # Select random samples
        sample_indices = np.random.choice(class_image_indices, 
                                        min(samples_per_class, len(class_image_indices)), 
                                        replace=False)
        
        for i, img_idx in enumerate(sample_indices):
            img_path = image_paths[img_idx]
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                axes[class_idx, i].imshow(img)
                axes[class_idx, i].axis('off')
                if i == 0:
                    axes[class_idx, i].set_title(f'{class_names[class_idx]}\n({img.shape[0]}x{img.shape[1]})', 
                                                fontsize=10, fontweight='bold')
                else:
                    axes[class_idx, i].set_title(f'{img.shape[0]}x{img.shape[1]}', fontsize=8)
    
    plt.suptitle('Sample Images from Dataset', fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

# Display samples
visualize_samples(image_paths, labels, class_names, samples_per_class=5)

## 1.3 Data Preprocessing

In [ ]:
# Import additional Keras utilities for data processing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Convert labels to numpy array
labels_array = np.array(labels)

# Apply debug mode limits if enabled
if config.debug_mode and len(image_paths) > config.debug_samples:
    print(f"⚠️ DEBUG MODE: Limiting dataset to {config.debug_samples} samples")
    # Stratified sampling to maintain class balance
    from sklearn.model_selection import train_test_split
    image_paths, _, labels_array, _ = train_test_split(
        image_paths, labels_array,
        train_size=config.debug_samples,
        stratify=labels_array,
        random_state=config.random_seed
    )
    n_samples = len(image_paths)
    print(f"   Using {n_samples} samples for debugging")

# Split data into train and validation sets using config
X_train_paths, X_val_paths, y_train, y_val = train_test_split(
    image_paths, labels_array, 
    test_size=config.train_test_split, 
    stratify=labels_array, 
    random_seed=config.random_seed
)

print(f"\n📊 Data Split:")
print(f"   Training samples: {len(X_train_paths)}")
print(f"   Validation samples: {len(X_val_paths)}")
print(f"   Split ratio: {(1-config.train_test_split)*100:.0f}/{config.train_test_split*100:.0f}")

print(f"\n📈 Class distribution in training set:")
train_counts = Counter(y_train)
for idx, class_name in enumerate(class_names):
    print(f"   {class_name}: {train_counts[idx]} images ({train_counts[idx]/len(y_train)*100:.1f}%)")
    
print(f"\n📈 Class distribution in validation set:")
val_counts = Counter(y_val)
for idx, class_name in enumerate(class_names):
    print(f"   {class_name}: {val_counts[idx]} images ({val_counts[idx]/len(y_val)*100:.1f}%)")

In [ ]:
# REQUIRED: Document your split (updated from config)
train_test_ratio = f"{int((1-config.train_test_split)*100)}/{int(config.train_test_split*100)}"
train_samples = len(X_train_paths)
test_samples = len(X_val_paths)

In [ ]:
print(f"\nTrain/Test Split: {train_test_ratio}")
print(f"Training Samples: {train_samples}")
print(f"Test Samples: {test_samples}")

In [ ]:
# Create optimized data generators with augmentation
def create_data_generators(train_paths, train_labels, val_paths, val_labels, 
                          img_size=(224, 224), batch_size=32):
    """
    Create training and validation data generators with optimizations
    """
    # Data augmentation for training
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2,
        fill_mode='nearest'
    )
    
    # Only rescaling for validation (no augmentation)
    val_datagen = ImageDataGenerator(rescale=1./255)
    
    # Create DataFrames for flow_from_dataframe
    train_df = pd.DataFrame({
        'filename': train_paths,
        'class': train_labels.astype(str)
    })
    
    val_df = pd.DataFrame({
        'filename': val_paths,
        'class': val_labels.astype(str)
    })
    
    # Create generators
    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        x_col='filename',
        y_col='class',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
        seed=config.random_seed
    )
    
    val_generator = val_datagen.flow_from_dataframe(
        val_df,
        x_col='filename',
        y_col='class',
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, val_generator

# Create generators using config settings
print("\n🔄 Creating data generators...")

# Use different batch sizes for custom CNN and transfer learning if needed
# For now, we'll use custom_batch_size as default
train_generator, val_generator = create_data_generators(
    X_train_paths, y_train, 
    X_val_paths, y_val,
    img_size=config.img_size,
    batch_size=config.custom_batch_size
)

print(f"✅ Data generators created:")
print(f"   Training batches: {len(train_generator)}")
print(f"   Validation batches: {len(val_generator)}")
print(f"   Image size: {config.img_size}")
print(f"   Batch size: {config.custom_batch_size}")
print(f"   Classes: {train_generator.class_indices}")
print(f"\n⚡ Optimization features:")
print(f"   Mixed precision: {config.use_mixed_precision}")
print(f"   Prefetch buffer: {'AUTO' if config.prefetch_buffer == tf.data.AUTOTUNE else config.prefetch_buffer}")


================================================================================
PART 2: CUSTOM CNN IMPLEMENTATION (5 MARKS)
================================================================================

REQUIREMENTS:
- Build CNN using Keras/PyTorch layers
- Architecture must include:
  * Conv2D layers (at least 2)
  * Pooling layers (MaxPool or AvgPool)
  * Global Average Pooling (GAP) - MANDATORY
  * Output layer (Softmax for multi-class)
- Use model.compile() and model.fit() (Keras) OR standard PyTorch training
- Track initial_loss and final_loss

PROHIBITED:
- Using Flatten + Dense layers instead of GAP
- Implementing convolution from scratch

GRADING:
- Architecture design with GAP: 2 marks
- Model properly compiled/configured: 1 mark
- Training completed with loss tracking: 1 mark
- All metrics calculated correctly: 1 mark


## 2.1 Custom CNN Architecture Design

In [ ]:
def build_custom_cnn(input_shape, n_classes):
    """
    Build custom CNN architecture with Global Average Pooling
    
    Args:
        input_shape: tuple (height, width, channels)
        n_classes: number of output classes
    
    Returns:
        model: compiled CNN model
    """
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import (Conv2D, MaxPooling2D, 
                                         GlobalAveragePooling2D, Dense,
                                         Dropout, BatchNormalization)
    
    model = Sequential([
        # Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Global Average Pooling (MANDATORY - replaces Flatten)
        GlobalAveragePooling2D(),
        
        # Dense layer
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        # Output layer
        Dense(n_classes, activation='softmax')
    ], name='Custom_CNN')
    
    return model

# Create model instance
print("\n🏗️ Building Custom CNN...")
custom_cnn = build_custom_cnn(tuple(config.img_size) + (3,), n_classes)

# Display model summary
custom_cnn.summary()

print(f"\n✅ Custom CNN created")
print(f"   Total parameters: {custom_cnn.count_params():,}")
print(f"   Uses Global Average Pooling: ✓")

### Compile Model

In [ ]:
# Compile the model
from tensorflow.keras.optimizers import Adam

custom_cnn.compile(
    optimizer=Adam(learning_rate=config.custom_learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Custom CNN compiled")
print(f"   Optimizer: {config.custom_optimizer}")
print(f"   Learning rate: {config.custom_learning_rate}")
print(f"   Loss function: categorical_crossentropy")

## 2.2 Train Custom CNN

In [ ]:
print("\n" + "="*70)
print("CUSTOM CNN TRAINING")

In [ ]:
# Initialize Training Manager for checkpointing and resumable execution
custom_cnn_manager = TrainingManager('custom_cnn', config)
progress_tracker.start_model('Custom CNN')

# Track training time
custom_cnn_start_time = time.time()

# Check if we should resume from checkpoint
initial_epoch = 0
if custom_cnn_manager.should_resume():
    print(f"\n🔄 Resuming training from epoch {custom_cnn_manager.get_initial_epoch()}")
    initial_epoch = custom_cnn_manager.get_initial_epoch()
    
    # Load model weights from checkpoint
    checkpoint_file = custom_cnn_manager.checkpoint_path / 'custom_cnn_best.h5'
    if checkpoint_file.exists():
        custom_cnn.load_weights(str(checkpoint_file))
        print(f"   Loaded weights from {checkpoint_file.name}")
else:
    print(f"\n🎯 Starting training from scratch")

print(f"   Total epochs: {config.custom_epochs}")
print(f"   Batch size: {config.custom_batch_size}")

In [ ]:
# Train the model with resumable execution
print(f"\n{'='*70}")
print("🏋️ TRAINING CUSTOM CNN")
print(f"{'='*70}\n")

try:
    # Train model
    history = custom_cnn.fit(
        train_generator,
        epochs=config.custom_epochs,
        initial_epoch=initial_epoch,
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=config.verbose_training
    )
    
    # Mark training as completed
    custom_cnn_manager.mark_completed()
    
    # Calculate training time
    custom_cnn_training_time = time.time() - custom_cnn_start_time
    
    # Track initial and final loss
    custom_cnn_initial_loss = history.history['loss'][0]
    custom_cnn_final_loss = history.history['loss'][-1]
    
    print(f"\n✅ Training completed successfully!")
    print(f"   Training time: {custom_cnn_training_time:.2f}s ({custom_cnn_training_time/60:.2f} min)")
    print(f"   Initial loss: {custom_cnn_initial_loss:.4f}")
    print(f"   Final loss: {custom_cnn_final_loss:.4f}")
    print(f"   Loss improvement: {((custom_cnn_initial_loss - custom_cnn_final_loss) / custom_cnn_initial_loss * 100):.2f}%")
    
    # Update progress tracker
    progress_tracker.complete_model('Custom CNN', custom_cnn_training_time)
    
except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user")
    print("   Progress has been saved. You can resume training by running this cell again.")
    custom_cnn_training_time = time.time() - custom_cnn_start_time
    custom_cnn_initial_loss = None
    custom_cnn_final_loss = None
    
except Exception as e:
    print(f"\n❌ Error during training: {str(e)}")
    raise

In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_title('Custom CNN: Model Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_title('Custom CNN: Model Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.title('Custom CNN: Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

## 2.4 Visualize Custom CNN Results

In [ ]:
# Evaluate Custom CNN on validation set
print(f"\n{'='*70}")
print("📊 EVALUATING CUSTOM CNN")
print(f"{'='*70}\n")

# Get predictions
y_pred_proba = custom_cnn.predict(val_generator, verbose=0)
y_pred = np.argmax(y_pred_proba, axis=1)
y_true = val_generator.classes

# Calculate metrics
custom_cnn_accuracy = accuracy_score(y_true, y_pred)
custom_cnn_precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
custom_cnn_recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
custom_cnn_f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print("✅ Custom CNN Performance:")
print(f"   Accuracy:  {custom_cnn_accuracy:.4f} ({custom_cnn_accuracy*100:.2f}%)")
print(f"   Precision: {custom_cnn_precision:.4f}")
print(f"   Recall:    {custom_cnn_recall:.4f}")
print(f"   F1-Score:  {custom_cnn_f1:.4f}")

# Detailed classification report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

## 2.3 Evaluate Custom CNN

In [ ]:
# Setup callbacks for checkpointing and early stopping
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard

callbacks = []

# Checkpoint callback - saves best model
if config.save_checkpoints:
    checkpoint_cb = ModelCheckpoint(
        filepath=str(custom_cnn_manager.checkpoint_path / 'custom_cnn_best.h5'),
        monitor='val_loss',
        save_best_only=config.save_best_only,
        save_weights_only=True,
        verbose=1
    )
    callbacks.append(checkpoint_cb)

# Early stopping - prevents overfitting
early_stop_cb = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)
callbacks.append(early_stop_cb)

# Reduce learning rate on plateau
reduce_lr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)
callbacks.append(reduce_lr_cb)

# TensorBoard for visualization (optional)
if config.log_metrics:
    tensorboard_cb = TensorBoard(
        log_dir=str(custom_cnn_manager.logs_path),
        histogram_freq=1
    )
    callbacks.append(tensorboard_cb)

print(f"\n✅ Callbacks configured:")
print(f"   - Model checkpointing: {config.save_checkpoints}")
print(f"   - Early stopping (patience=5)")
print(f"   - Learning rate reduction (patience=3)")
print(f"   - TensorBoard logging: {config.log_metrics}")


================================================================================
PART 3: TRANSFER LEARNING IMPLEMENTATION (5 MARKS)
================================================================================

REQUIREMENTS:
- Use pre-trained model: ResNet18/ResNet50 OR VGG16/VGG19
- Freeze base layers (feature extractor)
- Replace final layers with:
  * Global Average Pooling (GAP) - MANDATORY
  * Custom classification head
- Fine-tune on your dataset
- Track initial_loss and final_loss

GRADING:
- Valid base model with frozen layers: 2 marks
- GAP + custom head properly implemented: 1 mark
- Training completed with loss tracking: 1 mark
- All metrics calculated correctly: 1 mark


## 3.1 Load Pre-trained Model and Modify Architecture

In [ ]:
print("\n" + "="*70)
print("TRANSFER LEARNING IMPLEMENTATION")

In [ ]:
# Load pre-trained model (using config)
pretrained_model_name = config.pretrained_model
print(f"📦 Loading pre-trained model: {pretrained_model_name}")

In [ ]:
def build_transfer_learning_model(base_model_name, input_shape, n_classes):
    """
    Build transfer learning model with Global Average Pooling
    
    Args:
        base_model_name: string (ResNet50/VGG16/MobileNetV2)
        input_shape: tuple (height, width, channels)
        n_classes: number of output classes
    
    Returns:
        model: compiled transfer learning model with GAP
    """
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
    from tensorflow.keras.applications import ResNet50, VGG16, MobileNetV2
    
    # Load pre-trained model without top layers
    if base_model_name == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'VGG16':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif base_model_name == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError(f"Unsupported model: {base_model_name}")
    
    # Freeze base model layers
    base_model.trainable = False
    
    # Build custom head with Global Average Pooling
    x = base_model.output
    x = GlobalAveragePooling2D()(x)  # MANDATORY - replaces Flatten
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(n_classes, activation='softmax')(x)
    
    # Create model
    model = Model(inputs=base_model.input, outputs=outputs, name=f'TL_{base_model_name}')
    
    return model, base_model

# Create transfer learning model
print(f"\n🏗️ Building Transfer Learning Model with {pretrained_model_name}...")
transfer_model, base_model = build_transfer_learning_model(
    pretrained_model_name,
    tuple(config.img_size) + (3,),
    n_classes
)

# Display model summary
transfer_model.summary()

# Count frozen and trainable layers
frozen_layers = sum([not layer.trainable for layer in transfer_model.layers])
trainable_layers = sum([layer.trainable for layer in transfer_model.layers])
total_parameters = transfer_model.count_params()
trainable_parameters = sum([np.prod(v.get_shape()) for v in transfer_model.trainable_weights])

print(f"\n✅ Transfer Learning Model created:")
print(f"   Base Model: {pretrained_model_name}")
print(f"   Frozen Layers: {frozen_layers}")
print(f"   Trainable Layers: {trainable_layers}")
print(f"   Total Parameters: {total_parameters:,}")
print(f"   Trainable Parameters: {trainable_parameters:,}")
print(f"   Uses Global Average Pooling: ✓")

### Compile Transfer Learning Model

In [ ]:
# Compile the transfer learning model
from tensorflow.keras.optimizers import Adam

transfer_model.compile(
    optimizer=Adam(learning_rate=config.tl_learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("✅ Transfer Learning Model compiled")
print(f"   Optimizer: {config.tl_optimizer}")
print(f"   Learning rate: {config.tl_learning_rate}")
print(f"   Loss function: categorical_crossentropy")

## 3.2 Train Transfer Learning Model

In [ ]:
# Initialize Training Manager for Transfer Learning
tl_manager = TrainingManager('transfer_learning', config)
progress_tracker.start_model('Transfer Learning')

# Recreate generators with TL batch size
if config.tl_batch_size != config.custom_batch_size:
    print(f"\n🔄 Creating new generators with batch size {config.tl_batch_size}...")
    tl_train_generator, tl_val_generator = create_data_generators(
        X_train_paths, y_train,
        X_val_paths, y_val,
        img_size=config.img_size,
        batch_size=config.tl_batch_size
    )
else:
    print(f"\n♻️ Reusing existing generators")
    tl_train_generator = train_generator
    tl_val_generator = val_generator

# Track training time
tl_start_time = time.time()

# Training configuration from config
tl_learning_rate = config.tl_learning_rate
tl_epochs = config.tl_epochs
tl_batch_size = config.tl_batch_size
tl_optimizer = config.tl_optimizer

# Check if we should resume from checkpoint
initial_epoch = 0
if tl_manager.should_resume():
    print(f"\n🔄 Resuming training from epoch {tl_manager.get_initial_epoch()}")
    initial_epoch = tl_manager.get_initial_epoch()
    
    # Load model weights from checkpoint
    checkpoint_file = tl_manager.checkpoint_path / 'transfer_learning_best.h5'
    if checkpoint_file.exists():
        transfer_model.load_weights(str(checkpoint_file))
        print(f"   Loaded weights from {checkpoint_file.name}")
else:
    print(f"\n🎯 Starting training from scratch")

print(f"   Total epochs: {tl_epochs}")
print(f"   Batch size: {tl_batch_size}")

In [ ]:
# Setup callbacks for Transfer Learning
tl_callbacks = []

# Checkpoint callback
if config.save_checkpoints:
    tl_checkpoint_cb = ModelCheckpoint(
        filepath=str(tl_manager.checkpoint_path / 'transfer_learning_best.h5'),
        monitor='val_loss',
        save_best_only=config.save_best_only,
        save_weights_only=True,
        verbose=1
    )
    tl_callbacks.append(tl_checkpoint_cb)

# Early stopping
tl_early_stop_cb = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)
tl_callbacks.append(tl_early_stop_cb)

# Reduce learning rate
tl_reduce_lr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8,
    verbose=1
)
tl_callbacks.append(tl_reduce_lr_cb)

# TensorBoard
if config.log_metrics:
    tl_tensorboard_cb = TensorBoard(
        log_dir=str(tl_manager.logs_path),
        histogram_freq=1
    )
    tl_callbacks.append(tl_tensorboard_cb)

print(f"\n✅ Callbacks configured for Transfer Learning")

# Train Transfer Learning Model with resumable execution
print(f"\n{'='*70}")
print("🏋️ TRAINING TRANSFER LEARNING MODEL")
print(f"{'='*70}\n")

try:
    # Train model
    tl_history = transfer_model.fit(
        tl_train_generator,
        epochs=tl_epochs,
        initial_epoch=initial_epoch,
        validation_data=tl_val_generator,
        callbacks=tl_callbacks,
        verbose=config.verbose_training
    )
    
    # Mark training as completed
    tl_manager.mark_completed()
    
    # Calculate training time
    tl_training_time = time.time() - tl_start_time
    
    # Track initial and final loss
    tl_initial_loss = tl_history.history['loss'][0]
    tl_final_loss = tl_history.history['loss'][-1]
    
    print(f"\n✅ Training completed successfully!")
    print(f"   Training time: {tl_training_time:.2f}s ({tl_training_time/60:.2f} min)")
    print(f"   Initial loss: {tl_initial_loss:.4f}")
    print(f"   Final loss: {tl_final_loss:.4f}")
    print(f"   Loss improvement: {((tl_initial_loss - tl_final_loss) / tl_initial_loss * 100):.2f}%")
    
    # Update progress tracker
    progress_tracker.complete_model('Transfer Learning', tl_training_time)
    progress_tracker.show_overall_progress()
    
except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user")
    print("   Progress has been saved. You can resume training by running this cell again.")
    tl_training_time = time.time() - tl_start_time
    tl_initial_loss = None
    tl_final_loss = None
    
except Exception as e:
    print(f"\n❌ Error during training: {str(e)}")
    raise

In [ ]:
# Plot training history for Transfer Learning
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
axes[0].plot(tl_history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(tl_history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_title(f'{pretrained_model_name}: Model Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot accuracy
axes[1].plot(tl_history.history['accuracy'], label='Training Accuracy', linewidth=2)
axes[1].plot(tl_history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
axes[1].set_title(f'{pretrained_model_name}: Model Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Plot confusion matrix
tl_cm = confusion_matrix(tl_y_true, tl_y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(tl_cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=class_names, yticklabels=class_names)
plt.title(f'{pretrained_model_name}: Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

## 3.4 Visualize Transfer Learning Results

In [ ]:
# Evaluate Transfer Learning Model
print(f"\n{'='*70}")
print("📊 EVALUATING TRANSFER LEARNING MODEL")
print(f"{'='*70}\n")

# Get predictions
tl_y_pred_proba = transfer_model.predict(tl_val_generator, verbose=0)
tl_y_pred = np.argmax(tl_y_pred_proba, axis=1)
tl_y_true = tl_val_generator.classes

# Calculate metrics
tl_accuracy = accuracy_score(tl_y_true, tl_y_pred)
tl_precision = precision_score(tl_y_true, tl_y_pred, average='weighted', zero_division=0)
tl_recall = recall_score(tl_y_true, tl_y_pred, average='weighted', zero_division=0)
tl_f1 = f1_score(tl_y_true, tl_y_pred, average='weighted', zero_division=0)

print("✅ Transfer Learning Performance:")
print(f"   Accuracy:  {tl_accuracy:.4f} ({tl_accuracy*100:.2f}%)")
print(f"   Precision: {tl_precision:.4f}")
print(f"   Recall:    {tl_recall:.4f}")
print(f"   F1-Score:  {tl_f1:.4f}")

# Detailed classification report
print(f"\n📋 Detailed Classification Report:")
print(classification_report(tl_y_true, tl_y_pred, target_names=class_names, zero_division=0))

## 3.3 Evaluate Transfer Learning Model



================================================================================## 4.1 Metrics Comparison

PART 4: MODEL COMPARISON AND VISUALIZATION

================================================================================- Convergence behavior

- Model complexity

Compare both models on:- Training time
- Performance metrics

In [ ]:
print("\n" + "="*70)
print("MODEL COMPARISON")

In [ ]:
comparison_df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'Training Time (s)', 'Parameters'],
    'Custom CNN': [
        custom_cnn_accuracy,
        custom_cnn_precision,
        custom_cnn_recall,
        custom_cnn_f1,
        custom_cnn_training_time,
        0  # TODO: Fill with custom CNN total parameters
    ],
    'Transfer Learning': [
        tl_accuracy,
        tl_precision,
        tl_recall,
        tl_f1,
        tl_training_time,
        trainable_parameters
    ]
})

In [ ]:
print(comparison_df.to_string(index=False))

## 4.2 Visual Comparison


================================================================================
PART 5: ANALYSIS (2 MARKS)
================================================================================

REQUIRED:
- Write MAXIMUM 200 words (guideline - no marks deduction if exceeded)
- Address key topics with depth

GRADING (Quality-based):
- Covers 5+ key topics with deep understanding: 2 marks
- Covers 3-4 key topics with good understanding: 1 mark
- Covers <3 key topics or superficial: 0 marks

Key Topics:
1. Performance comparison with specific metrics
2. Pre-training vs training from scratch impact
3. GAP effect on performance/overfitting
4. Computational cost comparison
5. Transfer learning insights
6. Convergence behavior differences

## 5.1 Analysis

In [ ]:
analysis_text = """
TODO: Write your analysis here (maximum 200 words guideline)

Address:
1. Which model performed better and by how much?
   [Compare specific metrics]

2. Impact of pre-training vs training from scratch?
   [Discuss feature extraction, convergence speed]

3. Effect of Global Average Pooling?
   [Discuss parameter reduction, overfitting prevention]

4. Computational cost comparison?
   [Compare training time, total parameters]

5. Insights about transfer learning?
   [When to use transfer learning vs custom CNN]
"""

In [ ]:
# REQUIRED: Print analysis with word count
print("\n" + "="*70)
print("ANALYSIS")
print("="*70)
print(analysis_text)
print(f"\nAnalysis word count: {len(analysis_text.split())} words")
if len(analysis_text.split()) > 200:
    print("⚠️ Warning: Analysis exceeds 200 words (guideline)")
else:
    print("✅ Analysis within word count guideline")


================================================================================
PART 6: ASSIGNMENT RESULTS SUMMARY (REQUIRED FOR AUTO-GRADING)
================================================================================

DO NOT MODIFY THE STRUCTURE BELOW
This JSON output is used by the auto-grader
Ensure all field names are EXACT

## 6.1 Generate Results JSON

In [ ]:
def get_assignment_results():
    """
    Generate complete assignment results in required format
    
    Returns:
        dict: Complete results with all required fields
    """
    
    framework_used = "keras"  # TODO: Change to "pytorch" if using PyTorch
    
    results = {
        # Dataset Information
        'dataset_name': dataset_name,
        'dataset_source': dataset_source,
        'n_samples': n_samples,
        'n_classes': n_classes,
        'samples_per_class': samples_per_class,
        'image_shape': image_shape,
        'problem_type': problem_type,
        'primary_metric': primary_metric,
        'metric_justification': metric_justification,
        'train_samples': train_samples,
        'test_samples': test_samples,
        'train_test_ratio': train_test_ratio,
        
        # Custom CNN Results
        'custom_cnn': {
            'framework': framework_used,
            'architecture': {
                'conv_layers': 0,  # TODO: Count your conv layers
                'pooling_layers': 0,  # TODO: Count your pooling layers
                'has_global_average_pooling': True,  # MUST be True
                'output_layer': 'softmax',
                'total_parameters': 0  # TODO: Calculate total parameters
            },
            'training_config': {
                'learning_rate': 0.001,  # TODO: Your actual learning rate
                'n_epochs': 20,  # TODO: Your actual epochs
                'batch_size': 32,  # TODO: Your actual batch size
                'optimizer': 'Adam',  # TODO: Your actual optimizer
                'loss_function': 'categorical_crossentropy'  # TODO: Your actual loss
            },
            'initial_loss': custom_cnn_initial_loss,
            'final_loss': custom_cnn_final_loss,
            'training_time_seconds': custom_cnn_training_time,
            'accuracy': custom_cnn_accuracy,
            'precision': custom_cnn_precision,
            'recall': custom_cnn_recall,
            'f1_score': custom_cnn_f1
        },
        
        # Transfer Learning Results
        'transfer_learning': {
            'framework': framework_used,
            'base_model': pretrained_model_name,
            'frozen_layers': frozen_layers,
            'trainable_layers': trainable_layers,
            'has_global_average_pooling': True,  # MUST be True
            'total_parameters': total_parameters,
            'trainable_parameters': trainable_parameters,
            'training_config': {
                'learning_rate': tl_learning_rate,
                'n_epochs': tl_epochs,
                'batch_size': tl_batch_size,
                'optimizer': tl_optimizer,
                'loss_function': 'categorical_crossentropy'
            },
            'initial_loss': tl_initial_loss,
            'final_loss': tl_final_loss,
            'training_time_seconds': tl_training_time,
            'accuracy': tl_accuracy,
            'precision': tl_precision,
            'recall': tl_recall,
            'f1_score': tl_f1
        },
        
        # Analysis
        'analysis': analysis_text,
        'analysis_word_count': len(analysis_text.split()),
        
        # Training Success Indicators
        'custom_cnn_loss_decreased': custom_cnn_final_loss < custom_cnn_initial_loss if custom_cnn_initial_loss and custom_cnn_final_loss else False,
        'transfer_learning_loss_decreased': tl_final_loss < tl_initial_loss if tl_initial_loss and tl_final_loss else False,
    }
    
    return results

In [ ]:
# Generate and print results
try:
    assignment_results = get_assignment_results()
    
    print("\n" + "="*70)
    print("ASSIGNMENT RESULTS SUMMARY")
    print(json.dumps(assignment_results, indent=2))

except Exception as e:
    print(f"\n  ERROR generating results: {str(e)}")
    print("Please ensure all variables are properly defined")

In [ ]:
"""
ENVIRONMENT VERIFICATION - SCREENSHOT REQUIRED

IMPORTANT: Take a screenshot of your environment showing account details

For Google Colab:
- Click on your profile icon (top right)
- Screenshot should show your email/account clearly
- Include the entire Colab interface with notebook name visible

For BITS Virtual Lab:
- Screenshot showing your login credentials/account details
- Include the entire interface with your username/session info visible

Paste the screenshot below this cell or in a new markdown cell.
This helps verify the work was done by you in your environment.

"""

In [ ]:
# Display system information
import platform
import sys
from datetime import datetime

In [ ]:
print("ENVIRONMENT INFORMATION")
print("\n  REQUIRED: Add screenshot of your Google Colab/BITS Virtual Lab")
print("showing your account details in the cell below this one.")

In [ ]:
"""
FINAL CHECKLIST - VERIFY BEFORE SUBMISSION

□ Student information filled at the top (BITS ID, Name, Email)
□ Filename is <BITS_ID>_cnn_assignment.ipynb
□ All cells executed (Kernel → Restart & Run All)
□ All outputs visible
□ Custom CNN implemented with Global Average Pooling (NO Flatten+Dense)
□ Transfer learning implemented with GAP
□ Both models use Keras or PyTorch (NOT from scratch)
□ Both models trained with loss tracking (initial_loss and final_loss)
□ All 4 metrics calculated for both models
□ Primary metric selected and justified
□ Analysis written (quality matters, not just word count)
□ Visualizations created
□ Assignment results JSON printed at the end
□ No execution errors in any cell
□ File opens without corruption
□ Submit ONLY .ipynb file (NO zip, NO data files, NO images)
□ Only one submission attempt

"""